In [ ]:
import pandas as pd 

data = pd.read_csv("../dataset_full_texts.csv")
data.head()

In [ ]:
data.shape

In [4]:
df_examples = data.sample(20)

In [5]:
examples_text_list = df_examples["extracted_text"].to_list()
indexes = list(df_examples.index)

dict_ex_idx = dict(zip(indexes, examples_text_list))

In [6]:
example_text = examples_text_list[0]

## Automatic Data Annotation

In [7]:
with open("HF_TOKEN.txt", "r") as f:
        HF_TOKEN = f.read()

In [8]:
from transformers import AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

#### Prompt Functions

Important note: the API only supports a context length of 8k for this model, while the model may support 128k. Some random dude got around the problem by running a private endpoint and changing the 'Container Configuration', specifically the token settings to whatever length they required.

Apparent solution PAID: This part is relatively straightforward. Go to the the model card (e.g. https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct), Click on "Deploy" in the top right corner and select "Inference Endpoint". In the next page you can choose what hardware you want to run the model on, which will impact how much you will pay per hour. Set "Automatic Scale to Zero" to some value other than "never" to switch off the endpoint after X amount of time without request, so that you won't be paying for the endpoint while it's not in use. Then go to "Advanced Configuration" and set the maximum amount of tokens to whatever makes sense for your use case. With this procedure you will be able to make full use of the larger context windows of Llama 3 models.




In [9]:
def lazy_length_check(text, limit=8000, base_prompt_len=888):# 888 is the length of the current prompt

    chars = text.split(" ")
    len_text = len(chars)
    if len_text > limit - base_prompt_len: 
        print("Text over max. length. Applying padding.")
        padded_text = " ".join(chars[:limit - base_prompt_len])
        return padded_text 
    
    else:
        return text

In [10]:
def generate_prompt(text_to_classify):
    base_prompt = f"""<OBJECTIF_ET_PERSONA>
    Vous êtes un annotateur manuel. Votre tâche consiste à dire si un INPUT TEXT concerne un projet ou non.
    </OBJECTIF_ET_PERSONA>

    <INSTRUCTIONS>
    Suivre ces étapes :
    1. Déterminez si le INPUT TEXT concerne un projet ou non.
    2. Si le texte concerne un projet, renvoyez 1. Si le INPUT TEXT ne concerne pas un projet, renvoyez 0.
    3. Donnez la raisons pour votre réponse.
    4. N'écrivez aucun caractère d'échappement ni de guillemets.
    </INSTRUCTIONS>

    <FORMAT_DE_SORTIE>
    Le format de sortie doit être un dictionnaire json suivant ce modèle:
    {{"is_project": VOTRE RÉPONSE,
    "reason": VOTRE RÉPONSE}}
    </FORMAT_DE_SORTIE>

    <EXEMPLES>
    Voici quelques exemples :
    1. Exemple #1
    INPUT TEXT: "Pas moins de 200 tableaux peints par les impressionnistes à Bougival ont été répertoriés. Nombreux furent les peintres à s'installer pour un temps à Bougival ou à poser leur chevalet sur les rives de la Seine pour immortaliser paysage et vie en bord du fleuve. Lebourg (Alfred) 1849 – 1928 : attiré par la vallée de la Seine, Albert Lebourg peint à de nombreuses reprises le quai qui borde la Seine venant de Rueil. Il peint Bord de la Seine à Bougival en 1885. Monet (Claude) 1840 – 1926 : il s'installe à Bougival en 1869. Avec son tableau La Seine à Bougival ou le Pont de Bougival peint vers 1869/1870, Claude Monet a représenté l’ancien pont de la ville qui était à péage. Morisot (Berthe) 1841 – 1895 : elle passe plusieurs étés à Bougival de 1881 à 1884 dans la maison qui va devenir, en 2024, l’espace muséal Berthe Morisot. Elle peint à Bougival une quarantaine de toiles : La Fable (1883), Le Quai à Bougival (1883), Eugène Manet et sa fille dans le jardin (1883), Dans la Véranda (1884), Jardin à Bougival (1884), Roses trémières (1884), etc. Pissarro (Camille) 1830 – 1903 : installé à Louveciennes puis à Pontoise, Pissarro peint plusieurs toiles à Bougival : Maisons à Bougival (1870), Bords de la Seine à Bougival (1871), La Route de Louveciennes (1872). Renoir (Auguste) 1849 – 1928 : Renoir n’a jamais séjourné à Bougival, contrairement à son ami Monet, mais"
    {{"is_project": 0}}

    2. Exemple #2
    INPUT TEXT: "AMBITIONS DE LA ZAC : Une réponse aux enjeux contemporains Construit sur une friche industrielle et au contact d’un futur parc et du grand paysage, ce nouveau quartier répond à diérents enjeux : > Sociétaux : • Répondre aux besoins en logements de la commune, • Proposer un quartier avec une densité dénie pour ralentir l’étalement urbain, • Orir un accès aux services, espaces de santé, commerces et transports en commun, • Desservir le quartier par le réseau de transports en commun de la métropole (prolongation de la ligne 55), • Conserver l’identité culturelle et patrimoniale des lieux grâce au maintien des éléments industriels du site. > Environnementaux : • Renforcer la biodiversité de ce terrain malmené depuis 60 ans, en intégrant des plantations résistantes aux fortes chaleurs et la perméabilisation du site à 50 %. • Dépolluer les sols, • Respecter la charte Écoquartier : obtension de la médaille d’argent QDO en phase conception (Quartier Durable d’Occitanie). • Construire des bâtiments respectant les normes environnementales en vigueur, • Favoriser les modes de transport alternatifs à la voiture : création de pistes cyclables et de cheminements doux, d’une centralité orant des commerces et des activités, un accès à moins de 10 minutes à pied à la rive droite de la commune, • Préserver la limite entre ville et plaine cultivée. • Protéger les spots de biodiversités identiés dans l’étude d’impact (environ 7000 m2 ), • Orir un espace public paysager qualitatif avec près d’1 ha de parc urbain et de venelles vertes."
    {{"is_project": 1}}

    3. Exemple #3
    INPUT TEXT: "Rappel des procédures Suite à l’annulation du SCoT du Syndicat du Bassin d’Arcachon – Val de l’Eyre en date du 18 juin 2015 confirmé par la Cour Administrative d’Appel de Bordeaux en date du 28 décembre 2017, les élus des 17 communes du Bassin d’Arcachon et du Val de l’Eyre ont décidé collectivement de relancer la procédure en tenant compte de l’arrêt de la Cour Administrative d’Appel et des évolutions réglementaires. Procédure actuelle Par délibération du 9 juillet 2018, les élus du Conseil syndical du SYBARVAL ont prescrit l’élaboration d’un Schéma de Cohérence Territoriale à l’échelle du territoire en précisant les objectifs poursuivis et les modalités de concertation. Le SYBARVAL regroupe trois intercommunalités qui se situent autour ou à proximité du Bassin d’Arcachon : - la Communauté d’Agglomération du Bassin d’Arcachon Sud «COBAS » (Arcachon, GujanMestras, Le Teich, La Teste-de-Buch) ; - la Communauté d’Agglomération du Bassin d’Arcachon Nord «COBAN » (Andernos les Bains, Arès, Audenge, Biganos, Lanton, Lège-Cap-Ferret, Mios, Marcheprime) - la Communauté de Communes du Val de l’Eyre (Le Barp, Belin-Beliet, Lugos, Saint Magne, Salles)."
    {{"is_project": 1}}
    </EXEMPLES>

    <INPUT TEXT>
    {text_to_classify}
    </INPUT TEXT>
    """
    return base_prompt

In [ ]:
base_pr = """<OBJECTIF_ET_PERSONA>
    Vous êtes un annotateur manuel. Votre tâche consiste à dire si un INPUT TEXT concerne un projet ou non.
    </OBJECTIF_ET_PERSONA>

    <INSTRUCTIONS>
    Suivre ces étapes :
    1. Déterminez si le INPUT TEXT concerne un projet ou non.
    2. Si le texte concerne un projet, renvoyez 1. Si le INPUT TEXT ne concerne pas un projet, renvoyez 0.
    3. Donnez au moins deux raisons pour votre réponse.
    </INSTRUCTIONS>

    <FORMAT_DE_SORTIE>
    Le format de sortie doit être un dictionnaire json suivant ce modèle :
    {{"is_project": VOTRE RÉPONSE (0 ou 1),
    "reasons": VOTRE RÉPONSE}}
    </FORMAT_DE_SORTIE>

    <EXEMPLES>
    Voici quelques exemples :
    1. Exemple #1
    INPUT TEXT: "Pas moins de 200 tableaux peints par les impressionnistes à Bougival ont été répertoriés. Nombreux furent les peintres à s'installer pour un temps à Bougival ou à poser leur chevalet sur les rives de la Seine pour immortaliser paysage et vie en bord du fleuve. Lebourg (Alfred) 1849 – 1928 : attiré par la vallée de la Seine, Albert Lebourg peint à de nombreuses reprises le quai qui borde la Seine venant de Rueil. Il peint Bord de la Seine à Bougival en 1885. Monet (Claude) 1840 – 1926 : il s'installe à Bougival en 1869. Avec son tableau La Seine à Bougival ou le Pont de Bougival peint vers 1869/1870, Claude Monet a représenté l’ancien pont de la ville qui était à péage. Morisot (Berthe) 1841 – 1895 : elle passe plusieurs étés à Bougival de 1881 à 1884 dans la maison qui va devenir, en 2024, l’espace muséal Berthe Morisot. Elle peint à Bougival une quarantaine de toiles : La Fable (1883), Le Quai à Bougival (1883), Eugène Manet et sa fille dans le jardin (1883), Dans la Véranda (1884), Jardin à Bougival (1884), Roses trémières (1884), etc. Pissarro (Camille) 1830 – 1903 : installé à Louveciennes puis à Pontoise, Pissarro peint plusieurs toiles à Bougival : Maisons à Bougival (1870), Bords de la Seine à Bougival (1871), La Route de Louveciennes (1872). Renoir (Auguste) 1849 – 1928 : Renoir n’a jamais séjourné à Bougival, contrairement à son ami Monet, mais"
    Est-ce un projet: NO

    2. Exemple #2
    INPUT TEXT: "AMBITIONS DE LA ZAC : Une réponse aux enjeux contemporains Construit sur une friche industrielle et au contact d’un futur parc et du grand paysage, ce nouveau quartier répond à diérents enjeux : > Sociétaux : • Répondre aux besoins en logements de la commune, • Proposer un quartier avec une densité dénie pour ralentir l’étalement urbain, • Orir un accès aux services, espaces de santé, commerces et transports en commun, • Desservir le quartier par le réseau de transports en commun de la métropole (prolongation de la ligne 55), • Conserver l’identité culturelle et patrimoniale des lieux grâce au maintien des éléments industriels du site. > Environnementaux : • Renforcer la biodiversité de ce terrain malmené depuis 60 ans, en intégrant des plantations résistantes aux fortes chaleurs et la perméabilisation du site à 50 %. • Dépolluer les sols, • Respecter la charte Écoquartier : obtension de la médaille d’argent QDO en phase conception (Quartier Durable d’Occitanie). • Construire des bâtiments respectant les normes environnementales en vigueur, • Favoriser les modes de transport alternatifs à la voiture : création de pistes cyclables et de cheminements doux, d’une centralité orant des commerces et des activités, un accès à moins de 10 minutes à pied à la rive droite de la commune, • Préserver la limite entre ville et plaine cultivée. • Protéger les spots de biodiversités identiés dans l’étude d’impact (environ 7000 m2 ), • Orir un espace public paysager qualitatif avec près d’1 ha de parc urbain et de venelles vertes."
    Est-ce un projet: OUI

    3. Exemple #3
    INPUT TEXT: "Rappel des procédures Suite à l’annulation du SCoT du Syndicat du Bassin d’Arcachon – Val de l’Eyre en date du 18 juin 2015 confirmé par la Cour Administrative d’Appel de Bordeaux en date du 28 décembre 2017, les élus des 17 communes du Bassin d’Arcachon et du Val de l’Eyre ont décidé collectivement de relancer la procédure en tenant compte de l’arrêt de la Cour Administrative d’Appel et des évolutions réglementaires. Procédure actuelle Par délibération du 9 juillet 2018, les élus du Conseil syndical du SYBARVAL ont prescrit l’élaboration d’un Schéma de Cohérence Territoriale à l’échelle du territoire en précisant les objectifs poursuivis et les modalités de concertation. Le SYBARVAL regroupe trois intercommunalités qui se situent autour ou à proximité du Bassin d’Arcachon : - la Communauté d’Agglomération du Bassin d’Arcachon Sud «COBAS » (Arcachon, GujanMestras, Le Teich, La Teste-de-Buch) ; - la Communauté d’Agglomération du Bassin d’Arcachon Nord «COBAN » (Andernos les Bains, Arès, Audenge, Biganos, Lanton, Lège-Cap-Ferret, Mios, Marcheprime) - la Communauté de Communes du Val de l’Eyre (Le Barp, Belin-Beliet, Lugos, Saint Magne, Salles)."
    Est-ce un projet: OUI
    </EXEMPLES>

    <INPUT TEXT>
    </INPUT TEXT>
    """
print(len(base_pr.split(" ")))

In [12]:
def format_prompt(prompt):
    messages = [{"role": "user", "content": prompt}]
    messages_tokenized = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True, return_tensors="pt"
    )
    return messages_tokenized

In [13]:
def clean_response(output):
    clean_output_string = output.replace("\n", "").replace("\\", "")
    return clean_output_string

In [14]:
generation_params = dict(
    temperature=0.2,
    top_p=0.60,
    top_k=None,
    repetition_penalty=1.0,
    do_sample=True,
    max_new_tokens=512,
    return_full_text=False,
    seed=42,
    max_time=None,
    stream=False,
    use_cache=False,
    wait_for_model=False,
)

#### Querying

In [15]:
import requests
import huggingface_hub

def query(payload=None, api_url=None):
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()

headers = {"Authorization": f"Bearer {huggingface_hub.get_token()}"}
api_url = "https://api-inference.huggingface.co/models/" + model_id

#### Example

In [16]:
text = lazy_length_check(example_text)

prompt = generate_prompt(text)
prompt_formated = format_prompt(prompt)

output = query(payload={"inputs": prompt_formated, "parameters": {**generation_params}}, api_url=api_url)

In [ ]:
output

In [18]:
clean_output_string = clean_response(output[0]["generated_text"])

In [ ]:
clean_output_string

In [ ]:
import ast

output_dictionary = ast.literal_eval(clean_output_string)
print(output_dictionary)

In [ ]:
# verifying that we have the correct types to parse
print(type(output_dictionary))
print(type(output_dictionary["is_project"]))
print(type(output_dictionary["reason"]))
print(type(output_dictionary["reason"][0]))

### Data Annotation

In [21]:
def llm_annotate_dict(dict_ex_idx):
    
    for idx, text in dict_ex_idx.items():
        
        prompt = generate_prompt(text)
        prompt_formated = format_prompt(prompt)
        
        output = query(payload={"inputs": prompt_formated, "parameters": {**generation_params}}, api_url=api_url)
        
        clean_output_string = clean_response(output[0]["generated_text"])
        output_dictionary = ast.literal_eval(clean_output_string)
        
        output_dictionary["idx"] = idx
        
    return output_dictionary

In [22]:
def llm_annotate_df(text, verbose=True):
    
    text = lazy_length_check(text)
    
    prompt = generate_prompt(text)
    prompt_formated = format_prompt(prompt)
    
    output = query(payload={"inputs": prompt_formated, "parameters": {**generation_params}}, api_url=api_url)
    if verbose:
        print(output)
    
    clean_output_string = clean_response(output[0]["generated_text"])
    output_dictionary = ast.literal_eval(clean_output_string)
    
    is_project = output_dictionary["is_project"]
    reasoning = output_dictionary["reason"]
    
    return is_project, reasoning

In [ ]:
from numpy import NaN

# data["llm_is_project"] = NaN
# data["llm_reasoning"] = NaN

# how much annotation is left
data[data["llm_is_project"].isna()].shape

In [ ]:
data = pd.read_csv("annotated_data.csv")

In [24]:
import random
from tqdm import tqdm

tqdm.pandas()

index_data_no_annot = list(data[data["llm_is_project"].isna()].index)

while len(index_data_no_annot) != 0:
    
    print(len(index_data_no_annot), " left.")
    
    current_index = random.choice(index_data_no_annot)
    print("Current index ", current_index)
    
    current_text = data.iloc[current_index]["extracted_text"]
    print("Current text:\n", current_text[0:300])
    
    try:
        is_project, reasoning = llm_annotate_df(current_text)
        
        print("OUTPUT\n")
        print(is_project)
        print(reasoning)
        
        data.at[current_index, "llm_is_project"] = is_project
        data.at[current_index, "llm_reasoning"] = reasoning
        
        index_data_no_annot = list(data[data["llm_is_project"].isna()].index)
        
    except:
        print("Error parsing the LLM output")
    
    if current_index % 5 == 0:
        data.to_csv("annotated_data.csv", index=False)

15497  left.
Current index  14008
Current text:
 INFORMATION A LA POPULATION DE MONTESCOURT-LIZEROLLES DÉFINIR DES ZONES D'ACCÉLÉRATION DU DÉVELOPPEMENT DE PROJETS D'ÉNERGIES RENOUVELABLES (ZAER) À L'ÉCHELLE COMMUNALE La loi d'accélération de la production d'énergies renouvelables (AER) du 10 mars 2023 met les collectivités locales au cœur de la p
{'error': 'Rate limit reached. You reached free usage limit (reset daily). Please subscribe to PRO or Enterprise Hub to get a higher limit: https://hf.co/pricing'}
Error parsing the LLM output
15497  left.
Current index  16043
Current text:
 Conseil municipal du 28 mars 2023 Ordre du jour Information : Etat récapitulatif des indemnités des élus — année 2022 1. Approbation du procès-verbal du conseil municipal du 07 février 2023 2. Décisions municipales 3. Direction générale des services 3.1. 3.2. Sa3} 3.4, 3.5. 3.6. 3.7. 3.8. Approbatio
{'error': 'Rate limit reached. You reached free usage limit (reset daily). Please subscribe to PRO or Enter

KeyboardInterrupt: 